In [ ]:
!pip install --upgrade gspread

In [ ]:
# go to https://console.developers.google.com/
# click "Create project" (top right)
# click "enable APIS and Services" (top center)
# Enable the Google Sheets API for your project (select the Google Sheets option), then click "Enable"
# APIS & SERVICES (left) -> Credentials ->
# Create Credentials (Top) -> API key
# Create Credentials (Top) -> Service account -> name your service account -> create and continue -> Role = Owner -> Done
# Select service account -> Keys (Top center) -> Add key -> Create new key -> Download JSON

# share the sheet with the service account and all users

In [ ]:
import requests
import gspread
import json
import pandas as pd

In [ ]:
spreadsheet_id = input('write your spreadsheet id: ')

In [ ]:
spreadsheet_id

In [ ]:
range_name = 'Sheet1!A2:A2'
url = 'https://sheets.googleapis.com/v4/spreadsheets/'+spreadsheet_id+'/values/'+range_name

In [ ]:
response = requests.get(url)
response.json()

Handling secrets: secrets files, .gitignore and other prod tricks

In [ ]:
credentials_file = 'spreadsheet_secrets.txt'
handler = open(credentials_file, 'r')
lines = handler.readlines()
handler.close()

In [ ]:
API_key = lines[0]
params = {
   'key': API_key
}
headers = {
   'Content-Type': 'application/json'
}

In [ ]:
response = requests.get(url, headers=headers, params=params)

In [ ]:
response.json()

In [ ]:
json_credentials_path = 'ih-python-wrapers-f91e5ea6bcf8.json'

This is *not* how we would typically handle a request in production, for that we would use an more advanced authentication method, such as OAuth, which essentially generates Authentication keys with a timeout. However that is kind of a bother to handle, requiring even a call to *another* API to generate the keys

In [ ]:
with open(json_credentials_path) as json_file:
        credentials_data = json.load(json_file)
#credentials_data

In [ ]:
# Example, this is how you would generate a temporary access token...
# import requests
# import json
# from google.auth import crypt
# from google.auth import jwt

# def get_access_token(json_credentials_path):
#     url = 'https://oauth2.googleapis.com/token'
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {'grant_type': 'urn:ietf:params:oauth:grant-type:jwt-bearer','assertion': create_assertion(json_credentials_path)}
#     response = requests.post(url, headers=headers, data=data)
#     return response.json().get('access_token')

# def create_assertion(json_credentials_path):
#     with open(json_credentials_path) as json_file:
#         credentials_data = json.load(json_file)
#     credentials = jwt.Credentials.from_service_account_info(credentials_data,audience="https://sheets.googleapis.com/")
#     return credentials.signer.sign(b'')

# json_credentials_path = 'ironhackapiwrappersclass-133f259abbb8.json'
# access_token = get_access_token(json_credentials_path)

And then this access token times out after a few minutes and a new one has to be generated. It is a bother and we instead usually use a wrapper...

A wrapper handles the basic requests of an API in ways that are more functionally meaningful.

An API Wrapper can be used to add functionality that the API might not have itself, for example, by making one or more requests and templating the resulting (usually JSON) responses into a useful data structure appropriate for the language you are using (say a pandas dataframe, for example).

Another reason wrappers are used is to make it easier to use an API. Handling authentication, parameters, batching requests and generally abstracting away background admin we don't usually care so much about functionally.

In [ ]:
gc = gspread.service_account(filename=json_credentials_path)

In [ ]:
sheet = gc.open_by_key(spreadsheet_id)

In [ ]:
worksheet = sheet.get_worksheet(0)

In [ ]:
worksheet

In [ ]:
# get all values is not available in the original API (where you need to specify the range), but it's a natural call to make
worksheet.get_all_values()

In [ ]:
# or in a different form, which may be more helpful, but would not be available natively
print(worksheet.get_all_records())
pd.DataFrame(worksheet.get_all_records())

In [ ]:
# A final example
import re
worksheet.findall(re.compile('world'))

In [ ]:
# As another example, the native API has functions to add a row at once, which would require a loop and multiple requests if we wanted to add a bunch of rows,
# But since this a natural action to dowith the API

# Define the index and number of rows to insert; Also: new way to refer to sheets
worksheet = sheet.worksheet("Sheet2")
values = [['hello','is','it','me'],['I','live','in','multiple','rows']]

# Perform batch insertion
worksheet.insert_rows(values, row=2)

In [ ]:
# Popular Wrappers:

# https://spotipy.readthedocs.io/en/2.24.0/#api-reference [Spotify API Wrapper]
# https://github.com/yoavaviram/python-amazon-simple-product-api [Amazon product API Wrapper] 
# https://github.com/johnwmillr/LyricsGenius [Genius Music Lyrics API Wrapper]
# https://github.com/geopy/geopy [Geopy Geolocations and coordinates API Wrapper]
# https://github.com/aviaryan/python-gsearch [Google Search API Wrapper]


# List of Wrappers;
# https://github.com/realpython/list-of-python-api-wrappers?tab=readme-ov-file

